In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/california-housing-prices/housing.csv


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("camnugent/california-housing-prices")

# print("Path to dataset files:", path)



In [4]:
housing = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")

In [5]:
housing.drop("ocean_proximity" , inplace=True , axis = 1)
housing = housing.dropna(subset="total_bedrooms")


In [6]:
np.random.seed(42)
x = housing.drop("median_house_value" , axis = 1)
y = housing["median_house_value"]

housing.isna().sum()




longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split 
x_train , x_test , y_train , y_test = train_test_split(x, y , test_size= 0.2)

In [8]:
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(x_train , y_train)



Ridge()

In [9]:
model.score(x_train , y_train)

0.6360185691316951

In [10]:
model.score(x_test , y_test)

0.6400858817984223

# Lasso 

In [11]:
from sklearn.linear_model import Lasso 

model1 = Lasso()

model1.fit(x_train , y_train)

Lasso()

In [12]:
model1.score(x_train , y_train)

0.636018572207647

In [13]:
model1.score(x_test , y_test)

0.6400862985932623

## Ensemble Model :- Its a combination of smaller models to try and make better prediction than just a single model 

In [14]:
from sklearn.ensemble import RandomForestRegressor 

model1 = RandomForestRegressor()

model1.fit(x_train , y_train)

RandomForestRegressor()

In [15]:
model1.score(x_train , y_train)

0.9748097758655744

In [16]:
model1.score(x_test , y_test)

0.8214550319185272

### Prediction for a Regression Model 

In [17]:
y_preds = model1.predict(x_test)
np.array(y_preds[:5])

array([214237.01, 155280.  , 192216.  , 135481.  , 164132.  ])

In [18]:
np.array(y_test[:5])

array([245800., 137900., 218200., 220800., 170500.])

In [19]:
# Average difference between the predicted value and the true value 
# The output which is 32088.217729385862 tells on average this is the value that is 
# different from the true value just found the average of (y_preds - y_test)
from sklearn.metrics import mean_absolute_error 

mean_absolute_error(y_test , y_preds)

32088.217729385862

In [20]:
y_prediction = model1.predict(x_test)
y_prediction 

array([214237.01, 155280.  , 192216.  , ...,  90607.  , 108531.  ,
       111421.  ])

### Evaluating a Machine learning Model 

Three ways to evaluate scikit-learn models/estimators 

1. Estimator's built in `score` method 
2. The `scoring` parameter 
3. Problem specific metric functions 


## Regression Model Evaluation Metrics 

Types are :
1. R^2 (R - squared ) or coefficient of determination
2. Mean absolute Error (MAE)
3. Mean squared error (MSE)

## R^2:

what R^2 does is compares your model prediction to the mean of the targets. values can range from negative infinity (a very poor model) to 1 . For example all your model does is predict the mean of the targets , its R^2 value would be 0.
If your model perfectly predicts a range of numbers its R^2 value would be 1.


In [21]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

model.fit(x_train , y_train)

RandomForestRegressor()

In [22]:
from sklearn.metrics import r2_score 
r2_score(y_true = y_test ,y_pred = y_prediction)

0.8214550319185272

In [23]:
# Model performs worst everything is wrong 
r2_score(y_true = y_test , y_pred = np.full(len(y_test) , y_test.mean()))

0.0

In [24]:
# Model performs perfectly 
r2_score(y_true = y_test , y_pred = y_test)

1.0

**Mean Absolute Error (MAE):**

MAE is the average of the absolute differneces between predictions and actual values.

It gives you an idea about how wrong your model predictions are 


In [25]:
from sklearn.metrics import mean_absolute_error

y_prediction = model.predict(x_test)

mean_absolute_error(y_test , y_prediction)

31997.10869341815

In [26]:
df = pd.DataFrame(data = {"Actual values" : y_test ,
                          "Predicted values" : y_prediction,
                         })

df["differences"] = abs(df["Actual values"] - df["Predicted values"])

df["squared_differences"] = np.square(df["differences"])

df.head(10)



,Actual values,Predicted values,differences,squared_differences
14416,245800.0,213389.01,32410.99,1.050472e+09
16383,137900.0,154565.00,16665.00,2.777222e+08
7731,218200.0,189501.00,28699.00,8.236326e+08
1410,220800.0,140385.00,80415.00,6.466572e+09
1335,170500.0,164394.00,6106.00,3.728324e+07
16260,75000.0,73953.00,1047.00,1.096209e+06
18011,269400.0,281999.02,12599.02,1.587353e+08
10086,228900.0,209420.00,19480.00,3.794704e+08
18294,500001.0,459793.41,40207.59,1.616650e+09
18733,80800.0,72562.00,8238.00,6.786464e+07


In [27]:
#  This is what mean_absolute_error is calculating 
print(df["differences"].mean())
print(np.mean(np.absolute(y_test - y_prediction)))

# This is what mean_squared_error is calculating 
print(np.mean(df["squared_differences"]))

31997.10869341815
31997.10869341815
2453355545.218117


**Mean squared error**

In [28]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(y_test , y_prediction)

2453355545.218117